In [1]:
!pip install emoji
!pip install emosent-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 2.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=3ce98f01f139c38bf8df0bf21518dad02a3a3fca543dfb0a1b5ac4bac3014ac4
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
  Preparing metadata (setup.py) ... done
  Created wheel for emosent-py: filename=emosent_py-0.1.6-py3-none-any.whl size=28502 sha256=002821f94a816755e4f80b62c17e79ac099311e8fc5ffab053de649fd84928e8
  Stored in directory: /root/.cache/pip/wheels/32/37/bd/b4e67490f36c4beb85a1047d6cd13a356ffecbfa854eaf4688
Successfully built emosent-py
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
sys.path.append("../libs/leia/")
from leia import SentimentIntensityAnalyzer
import pandas as pd
import emoji
from emosent import get_emoji_sentiment_rank
from datetime import datetime

In [3]:
leia_classifier = SentimentIntensityAnalyzer()

In [4]:
def remove_duplicate_emoji(orig_str):
    prev_emoji = None
    remove_duplicate_emoji = []
    try:
        for c in orig_str:
            if c in emoji.EMOJI_DATA.keys():
                if prev_emoji == c:
                    continue
                prev_emoji = c
            remove_duplicate_emoji.append(c)
        return "".join(remove_duplicate_emoji)
    except:
        return orig_str

def check_emoji_sentiment(orig_str):
    neg = 0
    neu = 0
    pos = 0
    
    try:
        emoji_list = emoji.distinct_emoji_list(orig_str)
        if len(emoji_list) > 0:
            for item in emoji_list:
                
                try:
                    sentiment_rank = get_emoji_sentiment_rank(item)
                    neg += sentiment_rank['negative']
                    neu += sentiment_rank['neutral']
                    pos += sentiment_rank['positive']
                
                except:
                    pass
        
        tot = neg + neu + pos 
        
        return {
            'neg': round(neg/tot,3),
            'neu': round(neu/tot,3),
            'pos': round(pos/tot,3)
        }
    
    except:
        return {
            'neg': round(neg,3),
            'neu': round(neu,3),
            'pos': round(pos,3)
        }

def check_text_sentiment(orig_str):
    orig_str_without_emoji =  emoji.replace_emoji(remove_duplicate_emoji(orig_str), replace='')
    sentiment_polarity = leia_classifier.polarity_scores(orig_str_without_emoji)
    
    return {
            'neg': sentiment_polarity['neg'],
            'neu': sentiment_polarity['neu'],
            'pos': sentiment_polarity['pos']
    }

def compound_sentiment(text_sentiment, emoji_sentiment):
    compound = {
        'neg': 0,
        'neu': 0,
        'pos': 0
    }
    tot = 0
    for kind in compound.keys():
        compound[kind] = text_sentiment[kind] + emoji_sentiment[kind]
        tot += compound[kind]
    return {k: round(v / tot,3) for k, v in compound.items()}

In [5]:
df_geo_tweets = pd.read_csv('../src/application/data/tweets_brazil.csv', sep=',')

In [6]:
df_geo_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1505184 entries, 0 to 1505183
Data columns (total 14 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   text                 1501296 non-null  object 
 1   author_id            1501293 non-null  float64
 2   conversation_id      1501293 non-null  float64
 3   created_at           1501293 non-null  object 
 4   source               1501293 non-null  object 
 5   tweet_id             1501293 non-null  float64
 6   geo_id               1501293 non-null  object 
 7   retweet_count        1501293 non-null  float64
 8   reply_count          1501293 non-null  float64
 9   like_count           1501293 non-null  float64
 10  quote_count          1501293 non-null  float64
 11  in_reply_to_user_id  553669 non-null   float64
 12  referenced_type      431208 non-null   object 
 13  referenced_id        431208 non-null   float64
dtypes: float64(9), object(5)
memory usage: 160.8+ MB


In [7]:
df_geo_tweets.head()

,text,author_id,conversation_id,created_at,source,tweet_id,geo_id,retweet_count,reply_count,like_count,quote_count,in_reply_to_user_id,referenced_type,referenced_id
0,no gente cês parece retardado de querer saber ...,2.800738e+09,1.168307e+18,2019-09-01 23:38:37,Twitter for Android,1.168307e+18,d9d978b087a92583,0.0,1.0,3.0,0.0,NaN,NaN,NaN
1,q vírus é esse vei 🤬😡😡😡😡😡,1.010376e+18,1.168306e+18,2019-09-01 23:35:29,Twitter for Android,1.168306e+18,d9d978b087a92583,2.0,0.0,1.0,0.0,NaN,NaN,NaN
2,n posso toma duas corona q já fico feliz credo,2.969021e+09,1.168304e+18,2019-09-01 23:26:26,Twitter for Android,1.168304e+18,5722ff20ba67083b,0.0,2.0,1.0,0.0,NaN,NaN,NaN
3,era só uma corona geladinha agora poxa vida,6.329194e+07,1.168300e+18,2019-09-01 23:08:58,Twitter for Android,1.168300e+18,3b5c5c9c62f7c538,0.0,1.0,1.0,0.0,NaN,NaN,NaN
4,@SuavinBrito To bebendo corona de leve brota,1.610376e+08,1.168298e+18,2019-09-01 23:04:25,Twitter for Android,1.168299e+18,97bcdfca1a2dca59,1.0,1.0,1.0,0.0,3.431821e+09,replied_to,1.168298e+18


In [8]:
df_geo_id = pd.read_csv('../src/application/data/geolocation.csv', sep=',')

In [9]:
df_geo_id.head()

,country_code,place_type_name,place_type,geo_id,place_full_name,country
0,BR,Belo Horizonte,city,d9d978b087a92583,"Belo Horizonte, Brazil",Brazil
1,BR,Brasília,city,5722ff20ba67083b,"Brasília, Brazil",Brazil
2,BR,São João de Meriti,city,3b5c5c9c62f7c538,"São João de Meriti, Brasil",Brazil
3,BR,Rio de Janeiro,city,97bcdfca1a2dca59,"Rio de Janeiro, Brazil",Brazil
4,BR,Londrina,city,8aed51289a235ff4,"Londrina, Brasil",Brazil


In [10]:
df_geo_data = pd.merge(df_geo_tweets, df_geo_id, how="inner", on="geo_id")

In [11]:
df_geo_data['period'] = df_geo_data['created_at'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m') if isinstance(x, str) else 0
)

In [12]:
df_geo_data.groupby(['place_type_name', 'period']).size().reset_index(name="count").groupby(['place_type_name', 'period', 'count']).cumsum()

DataError: No numeric types to aggregate

In [ ]:
df_geo_data.groupby(['place_type_name'])['place_type_name'].count()\
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)